# Example usage of GraphQL API in python.
This notebook demonstrates the usage of a few of the GraphQL API endpoints.<br/>
If you'd like to explore the GraphQL API and it's schema in detail go to the [GraphQL Explorer](http://localhost:4000)

In [1]:
!pip install gql[all]

In [2]:
from gql import gql, Client
from gql.dsl import dsl_gql, DSLSchema, DSLQuery
from gql.transport.requests import RequestsHTTPTransport

In [3]:
# Select your transport with a defined url endpoint
transport = RequestsHTTPTransport(url="http://localhost:4000/")

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

### Retrieving data from the GraphQL graph
One can retrieve data from the GraphQL graph using `gql`

In [4]:
# Provide a GraphQL query
query = gql("""
    query Query($take: Int) {
      findManyZipcode(take: $take) {
        postcode
        latitude
        longitude
      }
    }
""")

# Execute the query on the transport
result = client.execute(query, variable_values={
  "take": 3
})
print(result)

{'findManyZipcode': [{'postcode': 6724, 'latitude': 39.4471, 'longitude': 79.0089}, {'postcode': 9465, 'latitude': -17.8169, 'longitude': -56.4559}, {'postcode': 4206, 'latitude': 23.7226, 'longitude': -114.303}]}


### Using DSLSchema instead of string-based GQL
Another way to retrieve data is by using the DSLSchema method. The method below retrieves the same data as the previous snippet but has a more developer-friendly API.

In [5]:
def convertDSLToQuery(schemaSelection: DSLSchema) -> str:
    """Converts a DSL schema query to a GraphQL query string"""
    return dsl_gql(DSLQuery(schemaSelection))

In [6]:
# Load the schema from the client
ds = DSLSchema(client.schema)

client.execute(convertDSLToQuery(
    ds.Query.findManyZipcode(take=3).select(
        ds.Zipcode.postcode,
        ds.Zipcode.latitude,
        ds.Zipcode.longitude,
    )
))

{'findManyZipcode': [{'postcode': 6724,
   'latitude': 39.4471,
   'longitude': 79.0089},
  {'postcode': 9465, 'latitude': -17.8169, 'longitude': -56.4559},
  {'postcode': 4206, 'latitude': 23.7226, 'longitude': -114.303}]}